In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY= os.getenv("LANGSMITH_API_KEY")

In [3]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

d:\learning\learn_langgraph_ss\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [6]:
from langchain_community.tools import DuckDuckGoSearchResults

In [7]:
search = DuckDuckGoSearchResults()

In [8]:
search.invoke("What is the weather in Bengaluru?")

"snippet: Bengaluru: The Garden City is set to experience light rainfall on Tuesday, May 13 2025. The city woke up at 05: 55 AM with cloudy skies this morning and it is likely to remain the same throughout ..., title: Bengaluru Weather Report: IMD Predicts Light Rainfall In The Garden ..., link: https://www.freepressjournal.in/india/bengaluru-weather-report-for-may-13, snippet: Today's Weather in Bengaluru: In Bengaluru today, the weather is expected to be Fair with a maximum temperature of 27°C and a minimum of 16°C. Sunrise in Bengaluru is set for 06:43 AM and ..., title: Bengaluru Weather Forecast 12 May 2025 - Times of India, link: https://timesofindia.indiatimes.com/weather/bengaluru-weather-forecast-today/560001, snippet: Bangalore's weather forecast for May 13, 2025, indicates a mix of sun and rain with temperatures ranging from 22.8°C to 35.7°C.The day presents an 84% chance of rain with 50% humidity and wind ..., title: Bangalore Weather Forecast: City may witness warm day wit

In [9]:
tools  = [search]

In [10]:
from langchain import hub

In [11]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [12]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [14]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
agent_executor.invoke({"input": "What is the weather in Delhi?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'weather in Delhi'}`




DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [19]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [20]:
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000, chunk_overlap=200
).split_documents(docs)

In [21]:
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [22]:
retriever.invoke("What is langgraph studio?")

[Document(id='81b45992-c8d0-4cdb-9d8c-b2bfc069a09d', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="0:00\n\n                            /0:20\n\n\n1×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConclusionBuilding agentic applications differs from traditional software development. While code editors remain important, new IDEs designed for agents are also needed. LangGraph Studio is a step in this direction, and we're excited to see how it enhances your workflow.For more on LangGraph Studio, check out our documentation. You can also watch a video walkthrough on YouTube if that's more your style. You can\xa0sign up for LangSmith\xa0today to try out LangGraph Studio for free.We'd also love your fee

In [24]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search for information about Langgraph. For any questions about Langgraph, you must use this tool!",
)

In [25]:
tools = [search, retriever_tool]

In [26]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [27]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input":"Hi! What is a langgraph studio and how to use it?"})



> Entering new AgentExecutor chain...

Invoking: `langgraph_studio` with `{'query': 'What is a langgraph studio and how to use it?'}`


is needed to make the development process easier. Building LLM applications differs from traditional software development, requiring different tooling outside of the traditional code editor.Coding is still important to developing LLM applications — after all, production-ready LangGraph applications have complicated custom logic in the nodes and edges of the graphs that are created. We don't aim to replace code editors but, instead, to augment the development experience with tools tailored for LangGraph applications. LangGraph Studio facilitates this by making it easy to visualize and interact with agent graphs, even if development still primarily happens in code. Visualizing graphs helps developers understand their structure. Furthermore, you can modify an agent result (or the logic underlying a specific node) halfway through the agent's trajectory. 

{'input': 'Hi! What is a langgraph studio and how to use it?',
 'output': 'LangGraph Studio is a tool designed to make developing agent applications easier.  It helps visualize and interact with agent graphs, even though the primary development still happens in code. \n\n\n\n\nLangGraph Studio is a desktop app, currently available for Apple Silicon. You can download a version here. \n'}

### ReAct Agent (Reasoning + Acting):

**Definition:** The ReAct framework combines reasoning and acting in a single loop to handle tasks. The agent uses natural language reasoning (thinking through steps) and task actions (performing tasks like calculations or data retrieval). The agent uses a combination of reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning.

**How it works:**
Step 1: The agent receives a question or task.
Step 2: It reasons aloud (in natural language) about how to solve it.